# Kalman filter for a simple linear system

### a) minimal state vector
$X_t=[x_t,\dot{x_t}]^T$

This is sufficient to model the system. Each successive step will depend on $X_{t-1}$ and the control. In the model, the control will be randomly set $\ddot{x_t}$.

$u_t = \ddot{x_t}$

$\overline{\mu_t} = A_t X_t + B_t u_t$

$\Sigma_t = R_t$

### b) Posterior of the prediction step

$p(x_t | u_t, x_{t-1}) = det(2\pi R_t)^{1/2}exp\{-\frac{1}{2}(x_t - A_{t-1}[x_{t-1},\dot{x_{t-1}}]^T - B u_t)^T  R_{t-1}^{-1} (x_t - A_{t-1}X_{t-1} - B u_t)\}$